In [1]:
from pathlib import Path
import tensorflow as tf
import numpy as np
import pandas as pd

from config import DATA_ROOT
from ecg_predict import batched_predict_all
from explainer import run_fused_pipeline_for_classes
from utils import import_key_data

In [2]:
ROOT = Path.cwd().parent

MODEL_PATH         = ROOT / "model" / "resnet_final.keras"
SNOMED_CLASSES_NPY = ROOT / "data" / "snomed_classes.npy"
SEL_DF_CSV         = ROOT / "data" / "ecg_model_pred_data.csv"

gender, age, labels, ecg_filenames = import_key_data(DATA_ROOT)

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
class_names = np.load(SNOMED_CLASSES_NPY, allow_pickle=True)

Indexing ECG headers: 100%|██████████| 43172/43172 [00:06<00:00, 6432.26it/s]


In [ ]:
probs = batched_predict_all(
    model,
    ecg_filenames,
    maxlen=5000,
    batch_size=32,
)

np.save("ecg_model_probs.npy", probs)


Predicting ECGs:  10%|▉         | 131/1347 [02:53<26:49,  1.32s/it]


KeyboardInterrupt: 

In [ ]:

sel_df = pd.read_csv(SEL_DF_CSV)


all_fused_payloads, df_lime_all, df_ts_all = run_fused_pipeline_for_classes(
    target_classes=["17338001"],
    sel_df=sel_df,
    model=model,
    class_names=class_names,
    max_examples_per_class=5,
    plot=True
)
